# Duality

Each canonical linear programming problem is associated with a minimisation problem called the **dual** problem. Recall the canonical problem:

<img src="figures/lecture-02/lec02-canonical-linear-prog.png" width="500" />


The primal problem and the dual problem are given as follows:

<img src="figures/lec03-primal-and-dual-problem.png" width="400" />


## Converting between Primal and Dual

We can convert a primal problem into its dual by creating a table and performing a transpose operation.

<img src="figures/lec03-ex2-primal-problem.png" width="300" />

First, we put values in a table and perform transpose.

\begin{equation*}
\begin{bmatrix}
\begin{array}{ccccc|c}
x_1 & x_2 & b \\ \hline 
1 & 0 & 30 \\ 
0 & 1 & 20 \\ 
1 & 2 & 54 \\ \hline 
2 & 3 & 1 \\ 
\end{array}
\end{bmatrix}
\Longrightarrow^{Transpose}
\begin{bmatrix}
\begin{array}{ccccc|c}
y_1 & y_2 & y_3 & c \\ \hline 
1 & 0 & 1 & 2 \\ 
0 & 1 & 2 & 3 \\ \hline 
30 & 20 & 54 & 1 \\ 
\end{array}
\end{bmatrix}
\end{equation*}

From the resulting, we can read off the values for the dual problem.

<img src="figures/lec03-ex2-dual-problem.png" width="400" />


## Duality Theorem

The duality theorem states that the solution of one problem can be used as the solution its dual problem.


<img src="figures/lec03-duality-theorem.png" width="600" />


First, let us find the solution to primal problem:

In [1]:
import linprog as lp

A = [[1, 0],
     [0, 1],
     [1, 2]]
b = [30, 20, 54]
c = [2, 3]

res = lp.maximise(A, b, c)
lp.pretty_print(res)

<IPython.core.display.Math object>

We see that the optimal solution is $\bar{x} = (30, 12)$ and the maximal value $f(\bar{x}) = 96$

Next, we find the solution to the dual problem:

In [2]:
import linprog as lp

A = [[1, 0, 1],
     [0, 1, 2]]
b = [2, 3]
c = [30, 20, 54]

res = lp.minimise(A, b, c)
lp.pretty_print(res)

<IPython.core.display.Math object>

The optimal $\bar{y} = (1/2, 0, 3/2)$ and $g(\bar{y}) = 96$. This means that $f(\bar{x}) = g(\bar{y}) = 96$.

## Duality and the Simplex Method

There is another important relationship between duality and the simplex method, which is expressed in the Duality Theorem:


<img src="figures/lec03-duality-theorem-c.png" width="600" />


Let us use the simplex method on our running example.

In [3]:
from simplex import SimplexSolver 

A = [[1, 0],
     [0, 1],
     [1, 2]]
b = [30, 20, 54]
c = [2, 3]

res = SimplexSolver().run_simplex(A, b, c, prob='max', enable_msg=False, latex_path='latex/lec03-ex2-max.tex')

\begin{equation*}
\begin{bmatrix}
\begin{array}{ccccc|c}
x_1 & x_2 & s_1 & s_2 & s_3 & b \\ \hline 
0 & 0 & 1/2 & 1 & -1/2 & 8 \\ 
0 & 1 & -1/2 & 0 & 1/2 & 12 \\ 
1 & 0 & 1 & 0 & 0 & 30 \\ \hline 
0 & 0 & 1/2 & 0 & 3/2 & 96 \\ 
\end{array}
\end{bmatrix}
\begin{array}{c}\\ 
s_2 \\ 
x_2 \\ 
x_1 \\ 
\\ 
\end{array}\end{equation*}

When the entries in the bottom row are non-negative then we can read off optimal solution of the dual problem:


<img src="figures/lec03-ex2-duality-simplex.png" width="400" />


## Economic Interpretation of the Dual Variables

The Dual Theorem tells us that $f(\bar{x})=g(\bar{y})$. By converting the primal problem to the dual problem, we found that $g(\bar{y}) = 30 y_1 + 20 y_2 + 54 y_3$ in our running example. The entries of $\bar{y}$ are called the **marginal values** of some resource unit $i$. Basically, $y_i$ is the amount of extra profit, we can expect to make for each additional unit of resource $i$ available. We can think $y_i$ as what is the fair price for one unit of resource $i$. If a marginal value is zero then there will be no increase in profit from increasing the number of resource $i$.

## Network Flow

A network flow problem can be reduced to a linear programming problem.

## Linear Programming and Matrix Games

A matrix game problem can be reduced to a linear programming problem and vice versa.

In [4]:
import numpy as np
from simplex import SimplexSolver 

def arr_to_str(array):
    return '[{}]'.format(', '.join(str(x) for x in array))

def solve_matrix_game(A):
    A = np.array(A)
    
    # Ensure that all entries are greater than 0.
    # If some of the entries are equal to or less than zero,
    # then add a fixed value k to each entry. This will not change
    # the optimal strategy for the two players. It will only add
    # an amount k to the value of the game.
    min_entry = A.min()
    k = 0
    if min_entry <= 0:
        k = min_entry * -1 + 1
    A += k
    
    # b and c are always ones
    b = np.ones(A.shape[0], dtype=np.int)
    c = np.ones(A.shape[1], dtype=np.int)
    
    # Solve using the simplex method
    solver = SimplexSolver()
    res = solver.run_simplex(A, b, c, prob='max', enable_msg=False, latex_path=None)
    
    # Compute player C's strategy, y_hat
    y_bar = np.array([res[k] for k in res.keys() if k.startswith('x_')])
    _lambda = np.array(y_bar).sum()
    y_hat = y_bar / _lambda
    
    # Compute player R's strategy, x_hat
    margin_values = solver.get_marginal_values()
    x_bar = np.array(margin_values)
    # x_delta = x_bar.sum()
    x_hat = x_bar / _lambda

    # Compute the value of the game
    # Substract the amount k added to the payoff matrix
    v = 1/_lambda - k

    print("Row player's strategy: {}".format(arr_to_str(x_hat)))
    print("Column player's strategy: {}".format(arr_to_str(y_hat)))
    print('Value of the game: {}'.format(v))
    #return x_hat, y_hat, v


In [5]:
A = [[-2, 1, 2],
     [ 3, 2, 0]]

solve_matrix_game(A)

Row player's strategy: [3/7, 4/7]
Column player's strategy: [2/7, 0, 5/7]
Value of the game: 6/7


In [6]:
A = [[-1, -1, -1],
     [ 1, -5, -5],
     [ 1,  5, 10]]

solve_matrix_game(A)

Row player's strategy: [0, 2/5, 3/5]
Column player's strategy: [1, 0, 0]
Value of the game: 1


In [7]:
A = [[1, 5, 3, 6],
     [4, 0, 1, 2]]

solve_matrix_game(A)

Row player's strategy: [3/5, 2/5]
Column player's strategy: [2/5, 0, 3/5, 0]
Value of the game: 11/5


In [8]:
from fractions import Fraction
A = [[0, 1],
     [1, Fraction(1,2)]]

solve_matrix_game(A)

Row player's strategy: [1/3, 2/3]
Column player's strategy: [1/3, 2/3]
Value of the game: 2/3
